In [1]:
#импорт библиотек позже будет больше явно 
import numpy as np
import cv2

In [2]:
car = list()
cars = [[0,0], [0,0], [0,0]]
x10, y10 = 590, 225

In [3]:
import math

def dr(cars):
	ax=cars[0][0]
	bx=cars[1][0]
	cx=cars[2][0]

	ay=cars[0][1]
	by=cars[1][1]
	cy=cars[2][1]

	ax_bx=ax-bx
	bx_cx=bx-cx
	cx_ax=cx-ax

	ay_by=ay-by
	by_cy=by-cy
	cy_ay=cy-ay

	axpbx=ax+bx
	bxpcx=bx+cx
	cxpax=cx+ax

	aypby=ay+by
	bypcy=by+cy
	cypay=cy+ay

	ax_bx=ax_bx**2
	bx_cx=bx_cx**2
	cx_ax=cx_ax**2

	ay_by=ay_by**2
	by_cy=by_cy**2
	cy_ay=cy_ay**2

	if (((ax_bx + ay_by) > (bx_cx + by_cy)) and ((ax_bx + ay_by) > (cx_ax + cy_ay))):
		dx=axpbx/2
		dy=aypby/2
		nx=cx
		ny=cy

	if (((bx_cx + by_cy) > (ax_bx + ay_by)) and ((bx_cx + by_cy) > (cx_ax + cy_ay))):
		dx=bxpcx/2
		dy=bypcy/2
		nx=ax
		ny=ay

	if (((cx_ax + cy_ay) > (ax_bx + ay_by)) and ((cx_ax + cy_ay) > (bx_cx + by_cy))):
		dx=cxpax/2
		dy=cypay/2
		nx=bx
		ny=by

	#print(dx)
	#print(dy)
	#print(nx)
	#print(ny)

	nx_dx = nx - dx
	ny_dy = ny - dy

	#print(nx_dx)
	#print(ny_dy)

	if (nx_dx == 0 and ny_dy > 0):
		grad=90
	elif (nx_dx == 0 and ny_dy < 0):
		grad =-90
	else:
		ngrad = ny_dy / nx_dx
		grad = math.atan(ngrad)
		grad = grad * 180
		grad = grad / math.pi

	if (nx_dx < 0 and ny_dy > 0):
		grad = 180 + grad
	elif (nx_dx < 0 and ny_dy < 0):
		grad = -180 + grad

	return dx, dy, grad

In [4]:
def tochka(x1, y1, x2, y2, x3, y3):
    a = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) #3
    b = math.sqrt((x2 - x3) ** 2 + (y2 - y3) ** 2) #1
    c = math.sqrt((x1 - x3) ** 2 + (y1 - y3) ** 2) #2
    return a, b, c

 
def areas(a, b, c):
    p = (a + b + c) / 2
    return math.sqrt(p * (p - a) * (p - b) * (p - c))
 
 
def main(x1, y1, x2, y2, x3, y3):
    
 
    a, b, c = tochka(x1, y1, x2, y2, x3, y3)
    p = areas(a, b, c)
    print(f'Площадь треугольника при а, b, c = {a}, {b}, {c} равна {p}')
    if max(a, b, c) == a:
        print(max(a, b, c))
        print(x3, y3)
        
    elif max(a, b, c) == b:
        print(max(a, b, c))
        print(x1, y1)
        
    elif max(a, b, c) == c:
        print(max(a, b, c))
        print(x2, y2)
        

In [5]:
# Получаем изображение !!!!!!! позже будет переделано на получение видеопотока с fps=1(возможно больше) !!!!!!
font = cv2.FONT_HERSHEY_COMPLEX
img2 = cv2.imread('polk.png', cv2.IMREAD_COLOR)

In [6]:
# Получили изображение ещё раз для контуров
# Перевод в чб для нахождения контуров
img = cv2.imread('polk.png', cv2.IMREAD_GRAYSCALE)
_, threshold = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY) #!!!!важно ограничения по цветам будут подобраны позже!!!!


In [7]:
# Находим контуры
contours, _= cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [8]:
# Пробегаем все контуры
for cnt in contours :
    approx = cv2.approxPolyDP(cnt, 0.009 * cv2.arcLength(cnt, True), True) #параметры подбираются позже
    # Во время пробежкт проверяем на треугольники
    if len(approx) == 3:      
        cv2.drawContours(img2, [approx], 0, (50, 205, 50), 5) 
        n = approx.ravel() 
        i = 0
        for j in n :
            if(i % 2 == 0):
                x = n[i]
                y = n[i + 1]

                # Координаты пока что в строку !!!  временно позже будет двумерный список но там небольшие проблемы !!!
                # для двумерного списка надо отличкать по цветам а этого пока что нет
                # Позже передача планируется в двумерном массиве
                # Сначала необходимо научиться различать роботов по цветам в общем то и потом там ну в общем достаточно легко
                
                
                string = str(x) + " " + str(y) 
                car.append(int(x))
                car.append(int(y))
                print(car)

                if(i == 0):# выбираем верхнюю координату от неё позже будем вести подсчёт, да не очень но потом там это поправитьcя
                    cv2.putText(img2, "Треугольник"+string, (x, y),
                                    font, 0.5, (0, 0, 0)) 
                else:
                    cv2.putText(img2, string, (x, y),
                              font, 0.5, (0, 0, 0)) 
            i += 1 

[250, 423]
[250, 423, 590, 225]
[250, 423, 590, 225, 927, 425]


In [9]:
for i in range(len(car)//6):
    cars[0][0] = car[0 + i]
    cars[0][1] = car[1 + i]
    cars[1][0] = car[2 + i]
    cars[1][1] = car[3 + i]
    cars[2][0] = car[4 + i]
    cars[2][1] = car[5 + i]
    x1, y1 = car[0 + i], car[1 + i]
    x2, y2 = car[2 + i], car[3 + i]
    x3, y3 = car[4 + i], car[5 + i]
    print(cars)
    print(dr(cars))
    print(main(x1, y1, x2, y2, x3, y3))
    
    print(dr(cars)[0])
    print(dr(cars)[1])

    

    x20, y20= dr(cars)[0], dr(cars)[1]
    print(x10, y10)
    ch = (x10 + x20)/2
    ch1 = (y10 + y20)/2
    print(ch, ch1) # 

[[250, 423], [590, 225], [927, 425]]
(588.5, 424.0, -89.56813044399395)
Площадь треугольника при а, b, c = 393.45139471096047, 391.8788077964921, 677.0029542033033 равна 67362.99999999996
677.0029542033033
590 225
None
588.5
424.0
590 225
589.25 324.5


In [10]:
cv2.circle(img2, (int(ch), int(ch1)), 10, (0,0,255), -1)

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [11]:
from PIL import Image
image = Image.open('polk.png')
 
w1, h1 = int(ch)-5, int(ch1)-5  # середина -5
w2, h2 = int(ch)+5, int(ch1)+5 # середина +5
rr, gg, bb = 0, 0, 0
 
for x in range(w1, w2):
    for y in range(h1, h2):
        r, g, b = image.getpixel((x, y))
        rr += r
        gg += g
        bb += b
 
cnt = (w2-w1) * (h2-h1)
print(rr//cnt, gg//cnt, bb//cnt)

34 177 76


In [ ]:
cv2.imshow('image2', img2) # и вуаля какой результат просто изумительно(на самом деле пока не очень)

# сначала вывод был с помощью mpl но там очень маленькая картинка и не удобно 
if cv2.waitKey(0) & 0xFF == ord('q'): 
    cv2.destroyAllWindows()